In [1]:
%pylab inline
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import brewer2mpl

Populating the interactive namespace from numpy and matplotlib


In [4]:
#read data
%run SC_dataframe.py
TS_df = pd.read_csv('Data/TS_df.csv')

In [5]:
Subjects = [503, 505, 508, 509, 512, 513, 516, 517, 518, 519, 523, 527, 528, 529, 530, 532, 534]
Conditions = ['FH', 'Fo', 'Fp', 'HF', 'Ho', 'Hp']
MotorMapping = [1, 2]
corr_df = []
corr_df = pd.DataFrame(columns=['Subj', 'Condition', 'FFA-M','FFA-NM', 'PPA-M', 'PPA-NM'], dtype=float)
# extract ts and correlate
i=1
for s, sub in enumerate(Subjects):
    for c, cond in enumerate(Conditions):           
        #motor mapping 1, RH-Face LH-Scene
        FFA_ts_m1 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 1)]['FFA'])
        PPA_ts_m1 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 1)]['PPA'])
        RH_ts_m1 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 1)]['RH'])
        LH_ts_m1 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 1)]['LH'])

        #motor mapping 2, RH-Scene, LH-Face
        FFA_ts_m2 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 2)]['FFA'])
        PPA_ts_m2 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 2)]['PPA'])
        RH_ts_m2 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 2)]['RH'])
        LH_ts_m2 = np.array(TS_df[(TS_df['Subject'] == sub) & (TS_df['Condition'] == cond) & (TS_df['MotorMapping'] == 2)]['LH'])
        
        FFA_ts = np.concatenate((FFA_ts_m1, FFA_ts_m2))
        PPA_ts = np.concatenate((PPA_ts_m1, PPA_ts_m2))

        #here use nan mask to deal with censored data
        FFA_ts[FFA_ts==0] = np.nan 
        PPA_ts[PPA_ts==0] = np.nan
        RH_ts_m1[RH_ts_m1==0] = np.nan
        LH_ts_m1[LH_ts_m1==0] = np.nan
        RH_ts_m2[RH_ts_m2==0] = np.nan
        LH_ts_m2[LH_ts_m2==0] = np.nan
        FFA_ts = np.ma.array(FFA_ts, mask=np.isnan(FFA_ts))
        PPA_ts = np.ma.array(PPA_ts, mask=np.isnan(PPA_ts))
        RH_ts_m1 = np.ma.array(RH_ts_m1, mask=np.isnan(RH_ts_m1))
        LH_ts_m1 = np.ma.array(LH_ts_m1, mask=np.isnan(LH_ts_m1))
        RH_ts_m2 = np.ma.array(RH_ts_m2, mask=np.isnan(RH_ts_m2))
        LH_ts_m2 = np.ma.array(LH_ts_m2, mask=np.isnan(LH_ts_m2))

        #correlate
                  
        corr_df.loc[i,'FFA-M'] = np.ma.corrcoef(FFA_ts, np.concatenate((RH_ts_m1, LH_ts_m2)))[0,1]
        corr_df.loc[i,'FFA-NM'] = np.ma.corrcoef(FFA_ts, np.concatenate((LH_ts_m1, RH_ts_m2)))[0,1]
        corr_df.loc[i,'PPA-M'] = np.ma.corrcoef(PPA_ts, np.concatenate((LH_ts_m1, RH_ts_m2)))[0,1]
        corr_df.loc[i,'PPA-NM'] = np.ma.corrcoef(PPA_ts, np.concatenate((RH_ts_m1, LH_ts_m2)))[0,1]

        corr_df.loc[i, 'Condition'] = cond
        corr_df.loc[i, 'Subj'] = sub
        i=i+1


In [6]:
corr_df.groupby('Condition').mean()

,Subj,FFA-M,FFA-NM,PPA-M,PPA-NM
Condition,,,,,
FH,519,-0.097346,-0.114651,-0.018394,-0.038902
Fo,519,-0.066943,-0.062595,-0.006924,-0.015857
Fp,519,-0.017107,-0.061615,-0.026381,-0.011802
HF,519,-0.086762,-0.088405,-0.072791,-0.050902
Ho,519,-0.059266,-0.051323,-0.079494,-0.057845
Hp,519,-0.004390,-0.003094,-0.053001,-0.025700
